In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn
mnist = input_data.read_data_sets('/tmp/data/',one_hot = True)

epoches = 10
classes = 10
batch_size = 128


x = tf.placeholder('float',[None,784])
print(x)
y = tf.placeholder('float')
print(y)

def conv(x,W):
    return tf.nn.conv2d(x,W,strides = [1,1,1,1],padding = 'SAME')

def maxpool(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1],padding='SAME' )

def convolutional_neural_network(x):
    weights_conv_1 = tf.Variable(tf.truncated_normal([2,2,1,8], stddev = 0.1))
    weights_conv_2 = tf.Variable(tf.truncated_normal([2,2,8,16], stddev = 0.1))
    weights_conv_3 = tf.Variable(tf.truncated_normal([2,2,16,32], stddev = 0.1))
    weights_full = tf.Variable(tf.truncated_normal([4*4*32, 1024], stddev = 0.1))
    weights_out = tf.Variable(tf.random_normal([1024,classes]))



    bias_conv_1 = tf.Variable(tf.zeros([8]))
    bias_conv_2 = tf.Variable(tf.zeros([16]))
    bias_conv_3 = tf.Variable(tf.zeros([32]))
    bias_full = tf.Variable(tf.zeros([1024]))
    bias_out = tf.Variable(tf.zeros([classes]))

    x = tf.reshape(x, shape = [-1,28,28,1])
    conv_1 = conv(x,weights_conv_1)
    conv_1 = maxpool(conv_1)
    print(conv_1.shape)

    conv_2 = conv(conv_1,weights_conv_2)
    conv_2 = maxpool(conv_2)
    print(conv_2.shape)
    conv_3 = conv(conv_2,weights_conv_3)
    conv_3 = maxpool(conv_3)
    print(conv_3.shape)
    
    conv_f = tf.reshape(conv_3,[-1,4*4*32])
    conv_f = tf.nn.relu(tf.matmul(conv_f,weights_full) + bias_full)
    conv_f = tf.nn.relu(conv_f)
    print(conv_f.shape)

    
    conv_o = tf.nn.relu(tf.matmul(conv_f,weights_out) + bias_out)
    return conv_o
    
    
def train(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction,labels = y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epoches):
            loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                #epoch_x = epoch_x.reshape((batch_size,chunks,chunk_size))
                
                _,c = sess.run([optimizer,cost],feed_dict = {x: epoch_x, y: epoch_y})
                loss += c
                
            print('Epoch',epoch,'complete out of',epoches,'loss:',loss)
            correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
            accuracy = tf.reduce_mean(tf.cast(correct,'float'))
            print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
train(x)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Tensor("Placeholder_2:0", shape=(?, 784), dtype=float32)
Tensor("Placeholder_3:0", dtype=float32)
(?, 14, 14, 8)
(?, 7, 7, 16)
(?, 4, 4, 32)
(?, 1024)
Epoch 0 complete out of 10 loss: 461.934378266
Accuracy: 0.6022
Epoch 1 complete out of 10 loss: 410.235436738
Accuracy: 0.604
Epoch 2 complete out of 10 loss: 402.779208481
Accuracy: 0.607
Epoch 3 complete out of 10 loss: 399.759190321
Accuracy: 0.6045
Epoch 4 complete out of 10 loss: 397.3995049
Accuracy: 0.6057
Epoch 5 complete out of 10 loss: 395.039312065
Accuracy: 0.6056
Epoch 6 complete out of 10 loss: 388.515327096
Accuracy: 0.6948
Epoch 7 complete out of 10 loss: 215.701276273
Accuracy: 0.7964
Epoch 8 complete out of 10 loss: 172.51652398
Accuracy: 0.8977
Epoch 9 complete out of 10 loss: 37.4889815261
Accuracy: 0.9859
